<a href="https://colab.research.google.com/github/tristanpadiou/Travel-assistant/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
torch.cuda.is_available()

False

In [53]:
from langchain_community.chat_models import ChatOllama

In [54]:
local_llm = 'llama3.2'
llama3 = ChatOllama(model=local_llm, temperature=0)

In [55]:
llama3.invoke('hello')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-03T21:15:22.9378707Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1532905900, 'load_duration': 1314825500, 'prompt_eval_count': 26, 'prompt_eval_duration': 145000000, 'eval_count': 10, 'eval_duration': 71000000}, id='run-839d969a-483b-434d-a1f5-43b3a9b6e188-0')

In [1]:
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, load_tools
from langchain.tools import Tool,tool,StructuredTool
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
)
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId

#structuring
import ast

from dataclasses import dataclass
from typing_extensions import TypedDict
from typing import Annotated, Literal
from pydantic import BaseModel, Field
#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

import pytz
from datetime import datetime
import os
import requests
import json
# import gradio as gr


c:\Users\tristan\localllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [56]:
llm = llama3

In [15]:
@tool
def trip_data_loader(tool_call_id: Annotated[str, InjectedToolCallId],filename: str) -> str:
  """
  Tool to load the trip_data, which is a pickle file.

  Include the extensions in filename. eg. file.pkl.
  """
  # with open(f'/content/drive/MyDrive/{filename}', 'rb') as f:
  #   trip_data=pickle.load(f)
  if filename=='trip_data.pkl':
    trip_data=trip_schedule
    return Command(update={'trip_data':str(trip_data),
                            'messages':[ToolMessage('uploaded the trip_data', tool_call_id=tool_call_id)
                                        ]})


In [16]:
@tool
def schedule_creator(tool_call_id: Annotated[str, InjectedToolCallId], schedule:str)->str:
  """Tool to create or add a schedule from the chat with the agent
  and then uses an llm to structure it.
  args: schedule - the schedule from the chat
  """
  result=llm.invoke(f'format this schedule: {str(schedule)} into a json format in the output, do not include ```json```, do not include comments either')
  return Command(update={'trip_data': ast.literal_eval(result.content),
                          'messages':[ToolMessage('added a schedule from the chat', tool_call_id=tool_call_id)
                                      ]})


In [17]:
# initializing time and date tool

#creating a schema
class time_tool_schema(BaseModel):
  continent: str = Field(description='continent')
  city: str = Field(description='city')

def date_time_tool(continent: str,city: str) -> str:
  """
  tool to get the current date and time in a city.

  """
  city=city.replace(' ','_')
  query=continent+'/'+city
  timezone = pytz.timezone(query)
  # Get the current time in UTC, and then convert it to the Marrakech timezone
  utc_now = datetime.now(pytz.utc)  # Get current time in UTC
  localized_time = utc_now.astimezone(timezone)  # Convert to Marrakech time
  time=localized_time.strftime('%Y-%m-%d %H:%M:%S')
  return time

current_date_time_tool=StructuredTool.from_function(name='current_date_time_tool', func=date_time_tool, description='To get the current date and time in any city',args_schema=time_tool_schema, return_direct=True)


In [18]:
@tool
def get_schedule(state: Annotated[dict, InjectedState])-> str:
  """
  Use this tool to get the information about the schedule once it has been loaded.
  args: none
  return: schedule
  """
  return state['trip_data']


In [19]:
# state
class State(TypedDict):
  """
  A dictionnary representing the state of the agent.
  """
  messages: Annotated[list, add_messages]
  trip_data: dict




In [20]:
@tool
def schedule_editor(query:str,state: Annotated[dict, InjectedState],tool_call_id: Annotated[str, InjectedToolCallId])-> str:
  """
  Tool to edit the schedule.
  Pass the query to the llm to edit the schedule.
  args: query - the query to edit the schedule.
  """
  file=state['trip_data']
  result=llm.invoke(f'Edit this schedule: {str(file)} following the instructions in the query: {query}, and include the changes in the schedule, but do not mention them specifically, only include the updated schedule json format in the output, do not include ```json```, do not include comments either')
  return Command(
                 update={'trip_data':ast.literal_eval(result.content),
                          'messages':[ToolMessage('edited the schedule', tool_call_id=tool_call_id)
                                      ]})

In [58]:
# langgraph
#loading tools
api_tools=load_tools(['wikipedia'])
langgraph_tools=[current_date_time_tool,get_schedule,trip_data_loader,schedule_creator, schedule_editor]+api_tools



graph_builder = StateGraph(State)

# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(langgraph_tools)

def chatbot(state: State):
  """ travel assistant that answers user questions about their trip.
  Depending on the request, leverage which tools to use if necessary."""
  return {"messages": [llm_with_tools.invoke(state['messages'])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=langgraph_tools)
graph_builder.add_node("tools", tool_node)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.set_entry_point("chatbot")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
memory=MemorySaver()
graph = graph_builder.compile(checkpointer=memory)



display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)



NotImplementedError: 

In [51]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content=f"hello, how are you ")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hello, how are you 


ValueError: Failed to parse a response from llama3.2:1b output: {}

 
 	      
   	            
       	            
      		
     	            
 	            
 	            
 	            
 	          
 	            
 	            
 	            
 	            


In [84]:
loll=graph.get_state(config).values["trip_data"]

In [85]:
loll

{'Monday': {'09:00': 'eating', '12:00': 'lunch', '18:00': 'sleep'},
 'Tuesday': {'09:00': 'fight', '12:00': 'flight', '18:00': 'sleep'}}

In [ ]:
def travel_agent(query,history):
  history_langchain_format = []
  for msg in history:
      if msg['role'] == "user":
          history_langchain_format.append(HumanMessage(content=msg['content']))
      elif msg['role'] == "assistant":
          history_langchain_format.append(AIMessage(content=msg['content']))
  history_langchain_format.append(HumanMessage(content=query))
  return graph.invoke({'messages':history_langchain_format})['messages'][-1].content

In [ ]:
travel_agent('What is planned janvier 2 in the schedule, the schedule is in the get_trip_schedule tool','')

"I cannot answer your question because the available tools do not allow me to access a trip schedule.  The `get_trip_schedule` tool is not working correctly.  It requires a parameter, but I don't know what that parameter should be."

In [ ]:
# interface=gr.ChatInterface(
#     travel_agent,
#     type='messages'
# )
# interface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b7614156361350b707.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b7614156361350b707.gradio.live
